In [6]:
# from llvmlite import ir
# import ast

from __future__ import print_function

import inspect
from ctypes import CFUNCTYPE, c_int, POINTER
import sys

try:
    from time import perf_counter as time
except ImportError:
    from time import time

import numpy as np

try:
    import faulthandler

    faulthandler.enable()
except ImportError:
    pass

import llvmlite.ir as ll
import llvmlite.ir as ir
import llvmlite.binding as llvm
import ast


llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()


t1 = time()

# Create some useful types
int_type = ir.IntType(32)
int_ptr_type = ir.PointerType(ir.IntType(32))
fnty = ir.FunctionType(ir.VoidType(), (int_ptr_type, int_ptr_type))

# Create an empty module...
module = ir.Module(name="bruv")
# and declare a function named "fpadd" inside it
func = ir.Function(module, fnty, name="add")

# Now implement the function
block = func.append_basic_block(name="entry")
builder = ir.IRBuilder(block)
a, b = func.args


def ast_to_builder(tree, builder, inputs: dict[str, any]):
    ast_to_ir = {
        ast.Add: builder.add,
        ast.Sub: builder.sub,
        ast.Mult: builder.mul,
        ast.Div: builder.sdiv,
    }
    print(ast_to_ir)
    for key in ast_to_ir:
        print(type(key))

    def rec(node):
        print("rec", node)
        if isinstance(node, ast.Name):
            return inputs[node.id]
        if isinstance(node, ast.BinOp):
            res = ast_to_ir[type(node.op)](rec(node.left), rec(node.right))
            print("here", type(res))
            return res
        if isinstance(node, ast.Constant):
            return ir.Constant(ir.IntType(32), node.value)
        raise RuntimeError(type(node), ast.dump(node))

    print(rec(tree.body[0].value))


tree = ast.parse(
    """
result = ((a + b) * (a + b)) - (5 / ((a + b) * (a + b)))
"""
)
print(ast.dump(tree, indent="\t"))
ast_to_builder(tree, builder, {"a": func.args[0], "b": func.args[1]})
builder.ret_void()

strmod = str(module)

print(strmod)

Module(
	body=[
		Assign(
			targets=[
				Name(id='result', ctx=Store())],
			value=BinOp(
				left=BinOp(
					left=BinOp(
						left=Name(id='a', ctx=Load()),
						op=Add(),
						right=Name(id='b', ctx=Load())),
					op=Mult(),
					right=BinOp(
						left=Name(id='a', ctx=Load()),
						op=Add(),
						right=Name(id='b', ctx=Load()))),
				op=Sub(),
				right=BinOp(
					left=Constant(value=5),
					op=Div(),
					right=BinOp(
						left=BinOp(
							left=Name(id='a', ctx=Load()),
							op=Add(),
							right=Name(id='b', ctx=Load())),
						op=Mult(),
						right=BinOp(
							left=Name(id='a', ctx=Load()),
							op=Add(),
							right=Name(id='b', ctx=Load()))))))],
	type_ignores=[])
{<class 'ast.Add'>: <bound method IRBuilder.add of <llvmlite.ir.builder.IRBuilder object at 0x7fe04c1a8730>>, <class 'ast.Sub'>: <bound method IRBuilder.sub of <llvmlite.ir.builder.IRBuilder object at 0x7fe04c1a8730>>, <class 'ast.Mult'>: <bound method IRBuilder.mul of <llvmlite.ir.builder.IRBui

In [7]:

t2 = time()

print("-- generate IR:", t2-t1)

t3 = time()

llmod = llvm.parse_assembly(strmod)

t4 = time()

print("-- parse assembly:", t4-t3)

# print(llmod)

pmb = llvm.create_pass_manager_builder()
pmb.opt_level = 3
pm = llvm.create_module_pass_manager()
pmb.populate(pm)

t5 = time()

pm.run(llmod)

t6 = time()

print("-- optimize:", t6-t5)

add = llmod.get_function("add")

print(dir(add))
for block in add.blocks:
  for instr in block.instructions:
    if str(instr.opcode) == "ret":
      
      def rec(instr):
        old = instr._kind
        if instr._kind == 'operand':
          instr._kind = 'instruction'
        print(instr, old)
        for operand in instr.operands:
          rec(operand)

      for arg in add.arguments:
        print(arg)
      print("return value")
      rec(instr)

t7 = time()

target_machine = llvm.Target.from_default_triple().create_target_machine()

with llvm.create_mcjit_compiler(llmod, target_machine) as ee:
    ee.finalize_object()
    cfptr = ee.get_function_address("add")
    print(cfptr)

-- generate IR: 1.8974512049462646
-- parse assembly: 0.0005949430633336306
-- optimize: 0.001060832990333438
['__bool__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_as_parameter_', '_capi', '_closed', '_dispose', '_kind', '_owned', '_parents', '_ptr', 'add_function_attribute', 'arguments', 'attributes', 'block', 'blocks', 'close', 'closed', 'detach', 'function', 'instruction', 'instructions', 'is_argument', 'is_block', 'is_declaration', 'is_function', 'is_global', 'is_instruction', 'is_operand', 'linkage', 'module', 'name', 'opcode', 'operands', 'storage_class', 'type', 'visibility']
i32 %.1
i32 %.2
return value
  ret void instruction
14